### Imports

In [45]:
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
import string
# nltk.download('punkt') # Download the 'punkt' tokenizer
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
import numpy as np
import gensim
import re
import pandas as pd
import os
from tabulate import tabulate
from gensim.models.phrases import Phrases, Phraser
nltk.download('stopwords')
import pickle
import itertools  #used for flattening lists of lists
import math
import csv
from help_functions import test_dictionary
from help_functions import get_next_qtr
from help_functions import company_quarter_lookup
import matplotlib.pyplot as plt



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ImportError: cannot import name 'company_quarter_lookup' from 'help_functions' (c:\Users\Steve\Documents\Grad School\Capstone\msds_capstone\help_functions.py)

## Load Stop Words, CSV Files, and Year List, SNL_Map

In [25]:
nltk_stop_words = set(stopwords.words('english'))
words_to_remove = ['below', 'haven']   #reasonable words to be meaningful
stop_words = list(filter(lambda word: word not in words_to_remove, nltk_stop_words))
print(stop_words)

### create list of csv files and years
csv_files = [os.path.join('data', file) for file in os.listdir('data')]
years_list = list(range(2009, 2025))
print('')
print(csv_files)

### create a look up file to get ciq id and map to related snl id
snl_ciq_map = {}
with open('snldata/SNl_CIQ_MAP.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader) #skips first line
    for row in reader:
        snl_ciq_map[int(row[1][2:])] = int(row[0])

### import csv as dataframes for transcript/company ids and loan data/snl ids
snl_df = pd.read_csv('snldata/snl_loans_v2.csv', encoding='utf-8')
transcript_id_df = pd.read_csv('snldata/transcript_snl_ciq_v2.csv', encoding='utf-8')

### Create blank dataframe to hold loan movement data
columns = ['transcript_id', 'ciq_id', 'snl_id', 'quarter', 'year','uncertainty_score','total_asset_before','total_asset_after',
           'loan_before','loan_after','l2a_delta']
delta_df = pd.DataFrame(columns=columns)

print('')
print(f'mapping files have been loaded in')

['that', 'only', 'can', 'wasn', 'ourselves', "you'll", 'o', 'any', 'll', "shan't", 'have', 't', "hadn't", 'after', 've', 'an', 'being', 'his', 'down', "isn't", 'all', 'no', "mightn't", "you're", 'why', "it's", 'we', 'both', "don't", 'be', 'by', 'hasn', 'mightn', 'hers', 'nor', 'doesn', 'at', 'theirs', 'just', 'he', 'so', 'doing', 'such', 'whom', 'over', 'from', 'yours', 'more', 'shouldn', 'needn', 'here', 'y', 'shan', 'does', "hasn't", 'or', 'didn', 'our', 'mustn', "wouldn't", "you've", 'has', 'in', 'its', 'some', 'how', 'through', 'wouldn', 'few', 'what', 'while', 'm', 'were', "mustn't", 'her', 'until', 'each', 's', "haven't", 'up', 'ours', 'are', 'she', 'was', 'had', 'because', 'you', 'most', 'not', 'aren', "needn't", 'between', 'before', "doesn't", 'been', 'your', 'couldn', 'ain', 'and', 'don', 're', "won't", 'am', 'will', "shouldn't", 'yourself', 'for', 'my', 'to', 'out', 'won', "she's", 'a', 'yourselves', 'myself', 'with', 'which', "that'll", 'of', "you'd", 'own', 'himself', 'then

In [31]:
transcript_id = 69064
row = transcript_id_df[transcript_id_df['TRANSCRIPTID'] == transcript_id]
print(f'row: {row}')
ciq_id =  row['COMPANYID'].values[0]
snl_id = row['INSTITUTIONID'].values[0]
# uncertainty_score = uncertainty_score_dict[transcript_id]
quarter = row['QUARTER'].values[0]
next_quarter = get_next_qtr(quarter)

print(f'CIQ ID: {ciq_id}, SNL ID: {snl_id}, quarter: {quarter}, nxt_qtr: {next_quarter}')
total_asset_before = snl_df[(snl_df['QUARTER'] == quarter) & (snl_df['INSTITUTIONID'] == snl_id)]['TOTAL_ASSETS'].values[0]

print(type(total_asset_before))

row:      COMPANYID  INSTITUTIONID   COMPANYNAME  QUARTER  \
107     253288        1011415  Cadence Bank  Q2 2010   

    DATE(E.MOSTIMPORTANTDATEUTC)  TRANSCRIPTID  
107                    7/23/2010         69064  
CIQ ID: 253288, SNL ID: 1011415, quarter: Q2 2010, nxt_qtr: Q3 2010


KeyError: 'TOTAL_ASSETS'

In [32]:
for index, row in transcript_id_df.iterrows():
    transcript_id = row['TRANSCRIPTID']
    ciq_id = row['COMPANYID']
    snl_id = row['INSTITUTIONID']
    date = row['DATE(E.MOSTIMPORTANTDATEUTC)']

    quarter = row['QUARTER']
    year = row['QUARTER'][-4:]
    next_quarter = get_next_qtr(quarter)

    try:
        total_asset_before = snl_df[(snl_df['QUARTER'] == quarter) & (snl_df['INSTITUTIONID'] == snl_id)]['TOTAL_ASSET'].values[0]
        total_asset_after = snl_df[(snl_df['QUARTER'] == next_quarter) & (snl_df['INSTITUTIONID'] == snl_id)]['TOTAL_ASSET'].values[0]

        loan_before = snl_df[(snl_df['QUARTER'] == quarter) & (snl_df['INSTITUTIONID'] == snl_id)]['LOANS'].values[0]
        loan_after = snl_df[(snl_df['QUARTER'] == next_quarter) & (snl_df['INSTITUTIONID'] == snl_id)]['LOANS'].values[0]

        l2a_delta = loan_after/total_asset_after - loan_before/total_asset_before
    except Exception as e:
        # print(f'loan calc error with {transcript_id}, date: {date}')
        continue

    new_row = {'transcript_id':transcript_id, 'ciq_id':ciq_id, 'snl_id':snl_id, 'quarter':quarter, 'year':year,
               'total_asset_before':total_asset_before,'total_asset_after':total_asset_after,'loan_before':loan_before,'loan_after':loan_after,
               'l2a_delta':l2a_delta}

    delta_df.loc[len(delta_df)] = new_row  # Adding at the next available index

    # print(f'SUCCESSFULLY ADDED ROW with {transcript_id}, date: {date}')

print(tabulate(delta_df.head(), headers='keys', tablefmt='pretty'))






loan calc error with 1514034, date: 7/25/2018
loan calc error with 64175, date: 4/29/2010
loan calc error with 1517644, date: 7/27/2018
loan calc error with 1548389, date: 7/26/2018
loan calc error with 30087, date: 7/23/2009
loan calc error with 1510755, date: 7/26/2018
loan calc error with 48069, date: 2/2/2010
loan calc error with 2900817, date: 7/28/2023
loan calc error with 1711336, date: 4/26/2019
loan calc error with 2019460, date: 4/29/2020
loan calc error with 1896963, date: 1/23/2020
loan calc error with 95545, date: 1/26/2011
loan calc error with 1844246, date: 10/25/2019
loan calc error with 117067, date: 4/21/2011
loan calc error with 1641483, date: 1/23/2019
loan calc error with 1894971, date: 1/23/2020
loan calc error with 1715582, date: 4/26/2019
loan calc error with 1574352, date: 10/22/2018
loan calc error with 45949, date: 10/26/2009
loan calc error with 752624, date: 1/30/2015
loan calc error with 1702087, date: 4/18/2019
loan calc error with 46712, date: 1/22/2010


In [33]:
print(tabulate(delta_df.head(), headers='keys', tablefmt='pretty'))

+---+---------------+----------+---------+---------+------+-------------------+--------------------+-------------------+-------------+------------+-----------------------+
|   | transcript_id |  ciq_id  | snl_id  | quarter | year | uncertainty_score | total_asset_before | total_asset_after | loan_before | loan_after |       l2a_delta       |
+---+---------------+----------+---------+---------+------+-------------------+--------------------+-------------------+-------------+------------+-----------------------+
| 0 |    1790209    | 13314302 | 4055785 | Q4 2019 | 2019 |        nan        |     12269288.0     |    12159919.0     | 10240434.0  | 10500284.0 | 0.028876319136914796  |
| 1 |    484381     | 2387628  | 4047200 | Q3 2012 | 2012 |        nan        |     16509440.0     |    18242878.0     | 11459931.0  | 14593134.0 |  0.10579181494653134  |
| 2 |    1177788    |  349262  | 102420  | Q3 2017 | 2017 |        nan        |     1763750.0      |     1776911.0     |  1465917.0  | 14698

In [44]:
delta_df.to_csv('transcript_loans.csv', index=False)

## Class Definition for Object Storage

In [22]:
class model_bundles:
  bundle_dict = {}

  def __init__(self, year, model, uncertainty_wordlist, corpus_bigrams, bigram_dict, flat_corpus_bigrams):

    self.year = year
    self.model = model
    self.uncertainty_wordlist = uncertainty_wordlist
    self.corpus_bigrams = corpus_bigrams
    self.bigram_dict = bigram_dict
    self.flat_corpus_bigrams = flat_corpus_bigrams

    model_bundles.bundle_dict[year] = self



## Functions

### Split Text Function

In [12]:
def split_text_to_sentences_words(text):
    # Split into sentences
    sentences = nltk.sent_tokenize(text)

    # Split each sentence into a list of words
    sentences_words = [nltk.word_tokenize(sentence) for sentence in sentences]

    return sentences_words

### Get Corpus, Process Data, and Create Bigrams

In [13]:
#takes single csv file and returns dictionary transcipt_id: [[word, word, word],[word, word, word]]
def get_corpus(csv_file):
   df = pd.read_csv(csv_file, encoding='utf-8')
   df['COMPONENTTEXT_SPLIT'] = df['COMPONENTTEXT'].apply(split_text_to_sentences_words)

   unprocessed_dict = {}
   for row in df.itertuples():
      if row.TRANSCRIPTID not in unprocessed_dict:
         unprocessed_dict[row.TRANSCRIPTID] = row.COMPONENTTEXT_SPLIT
      else:
         unprocessed_dict[row.TRANSCRIPTID].extend(row.COMPONENTTEXT_SPLIT)
   return unprocessed_dict

#takes unprocessed dict and returns processed dict
def process_the_data(unprocessed_dict, stop_words):
   num_tokens_before = 0
   num_tokens_after = 0
   processed_dict = {}

   for transcriptid, text in unprocessed_dict.items():
      p_text = []
      for sentence in text:
         p_sentence = []
         for word in sentence:
            num_tokens_before += 1
            p_word = word.lower()                                  #lowercase the text
            p_word = re.sub(r'(?<!\w)-(?!\w)|[^\w\s-]', '', p_word)  #remove punctuation but keep hyphens
            if p_word in stop_words or len(p_word) == 0:             #ignore if word has no length (ie was punctuation only) or in stop words
               continue
            p_sentence.append(p_word)
         p_text.append(p_sentence)
         num_tokens_after += len(p_sentence)
      processed_dict[transcriptid] = p_text
   return processed_dict


"""
below function creates:
corpus_bigrams (list of sentences for full corpus to run through word2vec
dict_bigrams: links transcript id to corpus for uncertainty calc
flat_corpus_bigrams: flat version of corpus bigrams for counting word appearances easily
"""

def create_bigrams(processed_dict, min_count=10, threshold = 100):
   processed_corpus = [['']]
   for value in processed_dict.values():
      processed_corpus.extend(value)

   phrases = Phrases(processed_corpus, min_count, threshold, scoring='default')
   bigram_phraser = Phraser(phrases)

   corpus_bigrams = [['']]

   dict_bigrams = {}
   for transcript_id, text in processed_dict.items():
      bigram_text = [bigram_phraser[sentence] for sentence in text]
      dict_bigrams[transcript_id] = bigram_text
      corpus_bigrams.extend(bigram_text)


   flat_corpus_bigrams = [item for sublist in corpus_bigrams for item in sublist]

   # print('bigrams created')
   return phrases, bigram_phraser, corpus_bigrams, dict_bigrams, flat_corpus_bigrams



### Word2Vec Model

In [19]:

def apply_word2vec(corpus_bigrams):
   model = gensim.models.Word2Vec (
    vector_size=150,    # Number of features in word vector

    window=10,   # Context window size (in each direction). Default is 5


    min_count=5, # Words must appear this many times to be in vocab.
                 #   Default is 5

    workers=10,  # Training thread count

    sg=1,        # 0: CBOW, 1: Skip-gram.

    hs=0,        # 0: Negative Sampling, 1: Hierarchical Softmax
                 #   Default is 0, NS

    negative=5   # Nmber of negative samples
                 #   Default is 5
   )

   model.build_vocab(
    corpus_bigrams,
    progress_per=20000  # Tweaks how often progress is reported
   )

   model.train(
    corpus_bigrams,
    total_examples=len(corpus_bigrams),
    epochs=10,        # How many training passes to take.
    report_delay=10.0 # Report progress every 10 seconds.
   )

   return(model)

## Run Models

In [23]:


for index, file in enumerate(csv_files):
    corpus_dict = get_corpus(file)
    processed_corpus_dict = process_the_data(corpus_dict, stop_words)
    phrases, bigram_phraser, corpus_bigrams, dict_bigrams, flat_corpus_bigrams = create_bigrams(processed_corpus_dict)
    trained_model = apply_word2vec(corpus_bigrams)

    similar_words = trained_model.wv.most_similar('uncertainty', topn=100)
    word_list = [word for word, number in similar_words]

    model_bundles(year=years_list[index], model=trained_model, uncertainty_wordlist=word_list,
               corpus_bigrams=corpus_bigrams, bigram_dict = dict_bigrams, flat_corpus_bigrams = flat_corpus_bigrams)




print('done!!')

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

Training the model...
 Modeling Training Done.

done!!


In [43]:
corpus_all_yrs = []

for year, bundle in model_bundles.bundle_dict.items():
    corpus_all_yrs.extend(bundle.corpus_bigrams)

all_yr_model = apply_word2vec(corpus_all_yrs)

sim_words = all_yr_model.wv.most_similar('uncertainty', topn=150)
word_list = [word for word, number in sim_words]

print(word_list)

Training the model...
 Modeling Training Done.

['geopolitical', 'uncertain', 'geopolitical_tensions', 'war_ukraine', 'fragile', 'unknown', 'unknowns', 'unpredictability', 'certainty', 'economic', 'gridlock', 'paralysis', 'instability', 'tensions', 'concerns', 'trade_wars', 'uncertainty_surrounding', 'debt_ceiling', 'stalling', 'strains', 'macro', 'nervousness', 'optimism', 'cautious', 'economy', 'trepidation', 'stalemate', 'uncertainly', 'swirling', 'unrest', 'tension', 'anxiety', 'microeconomic', 'fragility', 'contentious', 'posturing', 'volatility', 'dysfunction', 'depth_duration', 'rhetoric', 'fiscal_cliff', 'tenuous', 'miscalculation', 'unprecedented', 'watchword', 'decoupling', 'fluidity', 'cautiousness', 'apprehension', 'uncertainties', 'conflicts', 'murkier', 'lingering', 'prudence', 'macroeconomy', 'fears', 'concern', 'malaise', 'necessitates', 'probability_recession', 'political', 'ambiguity', 'sluggishness', 'elongation', 'deems', 'caution', 'recency', 'tightness', 'surround

In [37]:
# Empty list to hold all tokenized sentences
# Example of tokenized sentences across multiple documents
tokenized_sentences_doc1 = [['This', 'is', 'a', 'sentence'], ['Another', 'sentence']]
tokenized_sentences_doc2 = [['A', 'third', 'sentence'], ['More', 'tokens']]

# List of lists (each element is a list of tokenized sentences)
documents = [tokenized_sentences_doc1, tokenized_sentences_doc2]


all_tokenized_sentences = []

# Iterate over each document
for doc in documents:
    # Extend the list with the tokenized sentences from the document
    all_tokenized_sentences.extend(doc)

print(all_tokenized_sentences)


[['This', 'is', 'a', 'sentence'], ['Another', 'sentence'], ['A', 'third', 'sentence'], ['More', 'tokens']]


## Calc Uncertainty Scores

In [36]:
years_uncertainty_score_dict = {}

for year, bundle in model_bundles.bundle_dict.items():
    for transcript_id, bigrams in bundle.bigram_dict.items():
        transcript_text = list(itertools.chain.from_iterable(bigrams))

        neg_count = 0
        for word in transcript_text:
            if word in bundle.uncertainty_wordlist:
                neg_count += 1
                # if transcript_id == 3006899:
                #     print(word)
        neg_score = neg_count/len(transcript_text)

        years_uncertainty_score_dict[transcript_id] = neg_score




Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Steve\anaconda3\envs\capstone\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Steve\AppData\Local\Temp\ipykernel_13776\2305176017.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Steve\anaconda3\envs\capstone\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Steve\anaconda3\envs\capstone\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Steve\anaconda3\envs\capstone\Lib\site-packages\IPython\core\ultratb.py", line 1326, in structured_traceb

## Get Word Lists

In [ ]:
for key, value in model_year.years.items():

    print(f'{key}: {value.uncertainty_wordlist}')

In [ ]:
# idf_scores = {}
# n = len(years_list)

# for object in model_year.years.values():
#     for word in object.uncertainty_wordlist:
#         if word not in idf_scores.keys():
#             count = 0

#             for object in model_year.years.values():
#                 if word in object.flat_corpus_bigrams:
#                     count += 1



#         # print(f'{word}:({n}/{1+count})')
#         idf = math.log(n/(1+count))
#         idf_scores[word] = idf

# print(idf_scores)

idf_scores = {}
n = len(years_list)

for object in model_year.years.values():
    for word in object.uncertainty_wordlist:
        if word not in idf_scores.keys():
            count = 0

            for object in model_year.years.values():
                if word in object.uncertainty_wordlist:
                    count += 1

        print(f'{word}:({n}/{1+count})')
        idf = math.log(n/(1+count))
        idf_scores[word] = idf

# print(idf_scores)



In [ ]:
tfidf_df = empty_df = pd.DataFrame(index=years_list, columns=idf_scores.keys())
print(tfidf_df.head())

In [12]:
from collections import Counter


In [ ]:
combined_uncertainty_wordlist = []

for year in years_list:
    for uncertainty_word in model_year.years[year].uncertainty_wordlist:
        if uncertainty_word not in combined_uncertainty_wordlist:
            combined_uncertainty_wordlist.append(uncertainty_word)

print(len(combined_uncertainty_wordlist))

In [14]:
## count total instances of word
for year in years_list:
    word_counts = Counter((model_year.years[year].flat_corpus_bigrams))
    for uncertainty_word in combined_uncertainty_wordlist:
        count = word_counts[uncertainty_word]
        tfidf_df.at[year, uncertainty_word] = count

In [ ]:
print(years_list)

In [ ]:
for year in years_list:
    print(year)
    for word in model_year.years[year].uncertainty_wordlist:
        average_count = tfidf_df[word].mean()
        year_count = tfidf_df.at[year, word]

        if year_count/average_count > 3:
            print(f"{word} | {year_count} / {average_count}")
    print()
    print('---')

In [ ]:
index = model_year.years[2024].flat_corpus_bigrams.index('qt')
print(index)
start = max(0, index - 10)
end = min(len(word_list), index + 11)

print(model_year.years[2024].flat_corpus_bigrams[index - 10:index + 10])

In [30]:
for word in tfidf_df.columns:
    for year in years_list:
        idf = idf_scores[word]

        doc_length = len(model_year.years[year].uncertainty_wordlist)
        count = 0

        for checked_word in model_year.years[year].uncertainty_wordlist:
            if word == checked_word:
                count+=1

        tf = count/doc_length
        tf_idf = tf * idf

        tfidf_df.at[year, word] = tf_idf







In [ ]:
print(tfidf_df.head())

In [ ]:
# def top_10_columns_by_row(row):
#     # Sort values in descending order and get top 10 columns
#     return row.nlargest(10).index.tolist()

# df['top_10_columns'] = tfidf_df.apply(top_10_columns_by_row, axis=1)
# print(df[['top_10_columns']])

for index, row in tfidf_df.iterrows():
    # Sort row values in descending order and print top 10 with column names
    print(f"Row {index}:")
    sorted_row = row.sort_values(ascending=False)

    # Print column names and values for the top 10 items
    for col, value in sorted_row.head(20).items():
        print(f"{col}, Value: {value}")

In [ ]:
print(model_year.years[2010].corpus_bigrams[1:5])
print(len(model_year.years[2010].corpus_bigrams))

In [26]:
scores = {}

for year, model in model_year.years.items():
    score_list = [0] * len(model.uncertainty_wordlist)  #create dummy holder for frequency scores

    for index, word in enumerate(model.uncertainty_wordlist):
        count = 0
        for model in model_year.years.values():
            if word in model.uncertainty_wordlist:
                count += 1
        score_list[index] = count
    scores[year] = score_list


In [ ]:
for year, score_list in scores.items():
    print(year)
    for index, score in enumerate(score_list):
        if score < 2:
            print(f'{model_year.years[year].uncertainty_wordlist[index]} : {score}')
    print()
    print()

## Vectorizer if needed

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Example corpus: list of documents

# # Initialize TfidfVectorizer
# vectorizer = TfidfVectorizer(
#                                 lowercase=False,
#                                 max_features=1000000,
#                                 max_df=0.8,
#                                 min_df=5,
#                             )

# # Fit the vectorizer on the corpus and transform the corpus into a TF-IDF matrix
# tfidf_matrix = vectorizer.fit_transform(corpus_for_vectorizor)

# # Get the feature names (i.e., the words in the vocabulary)
# feature_names = vectorizer.get_feature_names_out()

# # Convert the TF-IDF matrix into a dense format (if necessary, for easier indexing)
# dense_matrix = tfidf_matrix.todense()

# # Convert dense matrix to a pandas DataFrame for easy lookup (optional but helpful)
# df_tfidf = pd.DataFrame(dense_matrix, columns=feature_names)

# # View the TF-IDF matrix
# print("TF-IDF matrix:\n", df_tfidf)

# # List of words you want to check the TF-IDF score for
# words_to_check = model_year.years[2021].uncertainty_wordlist

# # Loop through each word and print the corresponding TF-IDF score for each document
# for word in words_to_check:
#     if word in feature_names:
#         print(f"\nTF-IDF scores for word '{word}':")
#         print(df_tfidf[word])  # This will give you the TF-IDF score for that word in all documents
#     else:
#         print(f"Word '{word}' not found in the vocabulary")

In [ ]:
similar_words = trained_model.wv.most_similar('uncertainty', topn=100)
word_list = [word for word, number in similar_words]

# Print the most similar words and their similarity scores
for word, similarity in similar_words:
    print(f"{word}: Similarity = {similarity:.4f}")

## Calc Uncertainty Scores

In [26]:
directory = "data"
csv_files = [os.path.join(directory, file) for file in os.listdir(directory)]
# print(csv_files)

dfs = []
for file in csv_files:
    df = pd.read_csv(file, encoding='utf-8')
    dfs.append(df)
large_df = pd.concat(dfs, ignore_index=True)

In [ ]:
print(large_df.head())

In [ ]:


# Scatter plot of uncertainty score vs loan movement
plt.scatter(delta_df['uncertainty_score'], delta_df['l2a_delta'])
plt.xlabel('Uncertainty Score')
plt.ylabel('Loan Movement')
plt.title('Uncertainty Score vs Loan Movement')
plt.show()

correlation = delta_df['uncertainty_score'].corr(delta_df['l2a_delta'])
print(f"Correlation between Uncertainty Score and Loan Movement: {correlation}")